# 2.Linear Regression

## Data definition


#### 우선 data set이 필요하다. 그 data set을 기반으로 "예측"을 해야 하기 때문이다.

#### H(x) = Wx + b 
##### 위와 같은 형식의 가설을 통해 예측함. -> 선의 모양은 W와 b에 따라 달라진다.

#### Cost function : 가설과 data가 얼마나 다른가 확인. -> W와 b가 얼마나 정확한가?

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)    #난수 생성시 발생하는 문제를 이 한줄로 해결 가능하다.

## Data

In [3]:
#data set을 간단하게 가정하자.
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

In [4]:
# 3by1 tensor
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [5]:
#마찬가지로 3by1 tensor
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


## Weight Initialization

#### torch.zeros로 W와 B를 0으로 초기화. (ones였으면 1로 초기화됨)
#### requires_grad = True 를 해줌으로서 학습할 것을 명시해준다.

In [6]:
W = torch.zeros(1, requires_grad=True)
print(W)

tensor([0.], requires_grad=True)


In [7]:
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


## Hypothesis

### H(x) = Wx + b

In [8]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## Cost
##### 예측과 실제값의 차를 제곱한 것의 평균

In [9]:
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [10]:
print(y_train)

tensor([[1.],
        [2.],
        [3.]])


In [11]:
print(hypothesis - y_train)

tensor([[-1.],
        [-2.],
        [-3.]], grad_fn=<SubBackward0>)


In [12]:
print((hypothesis - y_train) ** 2)

tensor([[1.],
        [4.],
        [9.]], grad_fn=<PowBackward0>)


In [13]:
#mean은 평균구할 때 쓴다. 
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent
#### ->경사를 따라 내려가는 알고리즘.
#### ->cost fuction을 minimize한다.

In [14]:
#torch.optim 라이브러리 사용
#[w,b]는 학습할 tensor들, lr=0.01은 learning rate(얼마나 값을 변화시킬 것인가)
optimizer = optim.SGD([W, b], lr=0.01)

In [15]:
optimizer.zero_grad()    #zero_grad() 로 gradient 초기화
cost.backward()    #backward()로 gradient 계산
optimizer.step()    #step() 으로 개선

In [16]:
print(W)
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


In [17]:
# 가설이 얼마나 들어맞는지 확인
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.1333],
        [0.2267],
        [0.3200]], grad_fn=<AddBackward0>)


In [18]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(3.6927, grad_fn=<MeanBackward0>)


## Training with Full Code

In [19]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


## High-level Implementation with nn.Module

In [20]:
#fake data
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

##### linear regression 모델 만들기 -> PyTorch의 모든 모델은 nn.Module(제공됨)을 inherit해서 만들게 됨

In [21]:
class LinearRegressionModel(nn.Module):
    def __init__(self):    #사용할 레이어들 정의
        super().__init__()
        self.linear = nn.Linear(1, 1)    #linear regression 모델을 만들기 때문.

    def forward(self, x):
        return self.linear(x)

In [22]:
model = LinearRegressionModel()

## Hypothesis

In [23]:
#모델 생성해서 예측값 H(x) 구하기
hypothesis = model(x_train)
print(hypothesis)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward>)


## Cost

### PyTorch에서 MSE(mean squared error)제공. -> cost 구하기

In [24]:
print(hypothesis)
print(y_train)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward>)
tensor([[1.],
        [2.],
        [3.]])


In [25]:
cost = F.mse_loss(hypothesis, y_train)

In [26]:
print(cost)

tensor(2.1471, grad_fn=<MseLossBackward>)


## Gradient Descent

 ### 마지막에 주어진 cost를 이용해 w와 b를 바꾸어 cost를 줄인다.
 ### torch.optim에 있는 optimizer 이용가능.

In [27]:
# learning rate:0.01
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [28]:
#three steps
optimizer.zero_grad()
cost.backward()
optimizer.step()

## Training with Full Code

In [29]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.101, b: 0.508 Cost: 4.630286
Epoch  100/1000 W: 0.713, b: 0.653 Cost: 0.061555
Epoch  200/1000 W: 0.774, b: 0.514 Cost: 0.038037
Epoch  300/1000 W: 0.822, b: 0.404 Cost: 0.023505
Epoch  400/1000 W: 0.860, b: 0.317 Cost: 0.014525
Epoch  500/1000 W: 0.890, b: 0.250 Cost: 0.008975
Epoch  600/1000 W: 0.914, b: 0.196 Cost: 0.005546
Epoch  700/1000 W: 0.932, b: 0.154 Cost: 0.003427
Epoch  800/1000 W: 0.947, b: 0.121 Cost: 0.002118
Epoch  900/1000 W: 0.958, b: 0.095 Cost: 0.001309
Epoch 1000/1000 W: 0.967, b: 0.075 Cost: 0.000809


#### 점점 w와 b를 조정해서 cost, 즉 오차가 줄어듬을 알 수 있다.